# Predictions

In [17]:
# Imports
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.model_selection import StratifiedKFold, KFold


In [15]:
# Set random state for reproducibility
random_state = 199

In [8]:
# Read in geojson
bg_data = gpd.read_file("../data/block_groups.geojson")

In [12]:
# Group demographic columns together
registered = ['total_reg']
age = ['age_18_19', 'age_20_24', 'age_25_29','age_30_34','age_35_44', 'age_45_54', 'age_55_64', 'age_65_74','age_75_84', 'age_85over']
gender = [ 'gender_m', 'gender_f', 'gender_unknown'] 
# party = ['party_npp', 'party_dem', 'party_rep','party_lib', 'party_grn', 'party_con', 'party_ain', 'party_scl','party_oth', 'party_unk']
party = ['party_npp', 'party_dem', 'party_rep','party_lib', 'party_grn', 'party_con', 'party_ain', 'party_scl','party_oth']
ethnicity1 = ['eth1_eur', 'eth1_hisp', 'eth1_aa',
                'eth1_esa', 'eth1_oth', 'eth1_unk']
languages = ['lang_english', 'lang_spanish',
                'lang_portuguese',
                'lang_chinese', 'lang_italian',
                'lang_vietnamese', 'lang_other',
                'lang_unknown']

predictors = [*registered, *age, *gender, *party, *ethnicity1, *languages, 'mean_hh_income']


In [22]:
X = bg_data.set_index("GEOID20")[predictors]
y = bg_data.set_index("GEOID20")["2020_turnout_pct"]

In [23]:
X

,total_reg,age_18_19,age_20_24,age_25_29,age_30_34,age_35_44,age_45_54,age_55_64,age_65_74,age_75_84,...,eth1_unk,lang_english,lang_spanish,lang_portuguese,lang_chinese,lang_italian,lang_vietnamese,lang_other,lang_unknown,mean_hh_income
GEOID20,,,,,,,,,,,,,,,,,,,,,
250010121011,1248,19,68,81,76,171,192,224,215,164,...,124,623,15,4,1,1,0,5,601,96435.738782
250010121015,833,10,39,44,59,131,105,153,158,104,...,83,391,10,2,0,1,0,5,425,94926.003601
250010121013,720,4,34,43,45,103,85,149,141,89,...,51,390,2,2,0,2,0,1,326,96467.529167
250010121014,534,1,30,35,71,98,72,101,68,41,...,55,215,2,3,0,2,0,2,314,70610.726592
250277552022,774,14,67,34,34,104,135,151,119,88,...,96,426,2,0,0,3,0,8,338,105674.436693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250251601032,262,5,34,41,21,35,38,45,26,13,...,16,44,39,2,0,1,0,2,176,77150.660305
250251601024,168,6,18,15,16,36,25,26,22,3,...,10,42,20,0,0,0,2,3,102,83077.303571
250250711013,339,1,15,61,70,70,41,40,29,12,...,49,85,3,0,2,0,0,4,246,104028.268437


In [24]:
y

GEOID20
250010121011    0.804348
250010121015    0.782781
250010121013    0.792793
250010121014    0.654244
250277552022    0.845833
                  ...   
250251601032    0.687783
250251601024    0.655405
250250711013    0.808511
250251803013    0.881834
250250705022    0.725268
Name: 2020_turnout_pct, Length: 5078, dtype: float64

In [ ]:
# TO DO: CREATE K FOLDS WHILE STRATIFYING ON TOTAL VOTER REG
kf = KFold(n_splits=5,shuffle=True,random_state=random_state)
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=random_state)

In [ ]:
y_pred_results = {}
for i, (train_index, test_index) in enumerate(_.split(X,y)):
    X_train = X[train_index]
    X_test = X[test_index]
    